Testing a new calc for players selection using team means for matches home/guest and also position means. This method simplify the cartoleiro class method.

In [1]:
import cartoleiro

cart = cartoleiro.Cartoleiro(True)
df_scouts = cart.scout_table
df_rounds = cart.rounds_table
df_teams = cart.read_teams()

import pandas as pd

df_temp = df_scouts[["rodada_id", "atleta_id", "clube_id", "apelido", "posicao_id", "pontos_num", "jogos_num", "preco_num"]]
ind = df_temp.set_index(["atleta_id", "rodada_id"])
df_players = pd.DataFrame()
for player in ind.index.get_level_values(0).unique():
    # .copy(deep=True) to supress SettingWithCopyWarning
    df = df_temp[df_temp.atleta_id == player].copy(deep=True) 
    # test to flag the player who plays in a specific round
    df["jogou"] = (df["jogos_num"].diff() != 0) & (df["jogos_num"] != 0)
    df_players = pd.concat([df_players, df], axis=0)
# discard rows where player didnt play the match
df_scouts = df_players[df_players.jogou]

df_scouts = df_scouts.merge(df_teams[["id", "abreviacao"]], left_on="clube_id", right_on="id")

home = df_rounds[["rodada_id", "clube_casa_id"]].copy(deep=True) 
home.loc[:,"home_guest"] = len(home)*["H"]
df_scouts = df_scouts.merge(home, how="left", left_on=["rodada_id", "clube_id"], 
                            right_on=["rodada_id", "clube_casa_id"])
df_scouts["home_guest"] = df_scouts["home_guest"].fillna("G")

df_scouts = df_scouts.drop(["id", "jogou", "clube_casa_id"], axis=1)

df_scouts.head()
#df_scouts[["pontos_num"]].agg(["count","mean", "std", "min", "median", "max"])


,rodada_id,atleta_id,clube_id,apelido,posicao_id,pontos_num,jogos_num,preco_num,abreviacao,home_guest
0,1,81091,282,Jair,4,1.5,1,6.59,ATL,H
1,2,81091,282,Jair,4,2.0,2,6.07,ATL,G
2,3,81091,282,Jair,4,5.7,3,7.20,ATL,G
3,6,81091,282,Jair,4,2.5,4,6.73,ATL,G
4,10,81091,282,Jair,4,6.0,5,7.16,ATL,G


In [2]:
df_mean_pos = df_scouts.groupby(["posicao_id", "home_guest"])["pontos_num"].agg(["mean"])
df_mean_pos = df_mean_pos.drop(6) 
df_mean_pos = df_mean_pos.reset_index()
df_mean_pos.columns.values[2] = "idx_pos"
df_mean_pos

#df_mean_pos.pivot(index="posicao_id", columns="home_guest", values="mean").drop(6)



,posicao_id,home_guest,idx_pos
0,1,G,4.369192
1,1,H,4.225888
2,2,G,3.048449
3,2,H,3.932921
4,3,G,2.280569
5,3,H,3.450000
6,4,G,2.149849
7,4,H,2.729266
8,5,G,2.394783
9,5,H,3.600144


In [3]:
from cartolafc_api import cartola_api

df_market_min_matches = (pd.DataFrame(cartola_api.read_data()["atletas"]))
df_market = df_market_min_matches[["atleta_id", "status_id", "preco_num"]]
df_market = df_market[df_market.status_id == 7].drop("status_id", axis=1)

df_market_min_matches = df_market_min_matches[df_market_min_matches.jogos_num >= 5]
df_market_min_matches = df_market_min_matches[df_market_min_matches.status_id == 7]


df_market.head()


,atleta_id,preco_num
0,37646,4.98
5,37246,9.32
6,37673,4.23
9,37281,9.83
10,36940,6.97


In [4]:
cart.read_next_round()
next_round = cart.next_round
next_round = next_round[next_round["valida"] == True]
home_teams = list(next_round["clube_casa_id"])
guest_teams = list(next_round["clube_visitante_id"])

home_teams


[284, 354, 275, 276, 262, 266, 265, 293]

In [5]:
df_team_mean = df_scouts[df_scouts.atleta_id.isin(df_market["atleta_id"])].groupby([
    "clube_id","home_guest"])["pontos_num"].agg(["mean","median"])
df_team_mean = df_team_mean.reset_index()
df_team_mean = df_team_mean.pivot(index="clube_id", columns="home_guest", values="mean")
df_team_mean["idx_round"] = None
for team in home_teams: 
    df_team_mean.loc[team, "idx_round"] = df_team_mean.loc[team, "H"] 
for team in guest_teams: 
    df_team_mean.loc[team, "idx_round"] = df_team_mean.loc[team, "G"] 
        
df_team_mean = df_team_mean.drop(["G", "H"], axis=1).reset_index()
df_team_mean

home_guest,clube_id,idx_round
0,262,6.88689
1,263,3.08682
2,265,4.16033
3,266,2.91051
4,275,4.15171
5,276,4.52052
6,277,3.4628
7,283,2.05881
8,284,4.45938
9,285,2.60643


In [6]:
df_players_to_sel = df_scouts[df_scouts.atleta_id.isin(df_market_min_matches["atleta_id"])].groupby([
    "clube_id","atleta_id","posicao_id", "apelido", "abreviacao"])["pontos_num"].agg(["mean","median"])


df_players_to_sel["home_guest"] = None

for team in df_players_to_sel.index.get_level_values(0).unique():
    if team in home_teams:
        df_players_to_sel.loc[team,"home_guest"] = "H"
    else:
        df_players_to_sel.loc[team,"home_guest"] = "G"
        
df_players_to_sel = df_players_to_sel.reset_index()
df_players_to_sel = df_players_to_sel.merge(df_mean_pos, how="left", on=["posicao_id", "home_guest"])
df_players_to_sel = df_players_to_sel.merge(df_team_mean, how="left", on="clube_id")
df_players_to_sel = df_players_to_sel.merge(df_market[["atleta_id", "preco_num"]], how="left", on="atleta_id")
df_players_to_sel["idx_player"] = pd.to_numeric((1 * df_players_to_sel["mean"] + 2 * df_players_to_sel["median"] +
                                   df_players_to_sel["idx_pos"] + 2 * df_players_to_sel["idx_round"]) / 6)
df_players_to_sel.head()

,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
0,262,38144,2,Rafinha,FLA,4.411111,4.60,H,3.932921,6.88689,8.80,5.219636
1,262,38509,1,Diego Alves,FLA,4.875000,3.70,H,4.225888,6.88689,7.15,5.045779
2,262,38750,2,Filipe Luís,FLA,5.885714,6.60,H,3.932921,6.88689,13.77,6.132070
3,262,51772,4,Everton Ribeiro,FLA,4.031250,2.85,H,2.729266,6.88689,13.59,4.372383
4,262,71224,6,Jorge Jesus,FLA,5.889091,5.83,H,NaN,6.88689,14.49,NaN


In [7]:
print("GOLEIROS")

df_gk = df_players_to_sel[df_players_to_sel.posicao_id == 1]
df_gk.sort_values("idx_player", ascending=False).head(3)

GOLEIROS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
29,265,78584,1,Douglas Friedrich,BAH,7.273684,8.5,H,4.225888,4.16033,13.44,6.136705
17,263,74061,1,Gatito Fernández,BOT,5.217647,7.7,G,4.369192,3.08682,9.87,5.193413
154,341,86776,1,Jordi,CSA,7.278947,7.0,G,4.369192,2.57629,14.46,5.133452


In [8]:
print("ZAGUEIROS")

df_bw = df_players_to_sel[df_players_to_sel.posicao_id == 3]
df_bw.sort_values("idx_player", ascending=False).head(7)

ZAGUEIROS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
11,262,105897,3,Pablo Marí,FLA,5.211111,5.00,H,3.450000,6.88689,7.05,5.405816
5,262,73800,3,Rodrigo Caio,FLA,3.764286,3.45,H,3.450000,6.88689,8.35,4.648012
101,284,87342,3,Kannemann,GRE,4.433333,4.65,H,3.450000,4.45938,13.05,4.350347
34,265,92182,3,Juninho,BAH,3.910000,4.40,H,3.450000,4.16033,5.05,4.080110
110,285,79113,3,Víctor Cuesta,INT,6.242857,5.05,G,2.280569,2.60643,15.01,3.972714
117,290,73421,3,Rafael Vaz,GOI,4.992308,4.70,G,2.280569,3.5238,13.14,3.953414
106,285,63354,3,Rodrigo Moledo,INT,5.025000,5.35,G,2.280569,2.60643,7.65,3.869738


In [9]:
print("ATACANTES")

df_fw = df_players_to_sel[df_players_to_sel.posicao_id == 5]
df_fw.sort_values("idx_player", ascending=False).head(7)

ATACANTES


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
7,262,83257,5,Gabriel,FLA,10.793750,10.05,H,3.600144,6.88689,21.50,8.044613
99,284,86757,5,Everton,GRE,7.368750,5.95,H,3.600144,4.45938,19.05,5.297941
25,265,61188,5,Gilberto,BAH,7.481250,5.30,H,3.600144,4.16033,17.36,5.000342
10,262,90285,5,Bruno Henrique,FLA,6.100000,3.10,H,3.600144,6.88689,19.00,4.945655
131,293,91607,5,Rony,CAP,4.923077,4.70,H,3.600144,4.70526,11.42,4.555625
51,275,68920,5,Dudu,PAL,6.200000,4.10,H,3.600144,4.15171,18.02,4.383929
72,277,70360,5,Eduardo Sasha,SAN,6.394737,4.80,G,2.394783,3.4628,17.67,4.219188


In [10]:
print("LATERAIS")

df_sw = df_players_to_sel[df_players_to_sel.posicao_id == 2]
df_sw.sort_values("idx_player", ascending=False).head(5)

LATERAIS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
2,262,38750,2,Filipe Luís,FLA,5.885714,6.60,H,3.932921,6.88689,13.77,6.132070
65,276,78850,2,Reinaldo,SAO,6.805556,6.45,H,3.932921,4.52052,14.72,5.446586
0,262,38144,2,Rafinha,FLA,4.411111,4.60,H,3.932921,6.88689,8.80,5.219636
32,265,90444,2,Moisés,BAH,5.493750,6.60,H,3.932921,4.16033,13.83,5.157888
77,277,88065,2,Jorge,SAN,6.675000,6.80,G,3.048449,3.4628,15.03,5.041510


In [11]:
print("MEIAS")

df_mf = df_players_to_sel[df_players_to_sel.posicao_id == 4]
df_mf.sort_values("idx_player", ascending=False).head(7)

MEIAS


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
8,262,87863,4,Arrascaeta,FLA,11.616667,11.60,H,2.729266,6.88689,18.05,8.553286
9,262,89256,4,Gerson,FLA,4.280000,5.35,H,2.729266,6.88689,9.57,5.247175
168,354,96340,4,Lima,CEA,5.385714,6.60,H,2.729266,4.28355,6.70,4.980346
160,354,71844,4,Thiago Galhardo,CEA,6.441176,5.30,H,2.729266,4.28355,11.29,4.722923
98,284,81845,4,Alisson,GRE,5.760000,5.45,H,2.729266,4.45938,8.08,4.718003
6,262,78478,4,Willian Arão,FLA,4.761111,3.30,H,2.729266,6.88689,13.72,4.644027
121,290,90572,4,Léo Sena,GOI,5.033333,6.20,G,2.149849,3.5238,7.83,4.438465


In [12]:
#df_players_to_sel.info()
print("TÉCNICOS")
df_players_to_sel.groupby(["clube_id", "abreviacao"])["idx_player"].agg(["mean"]).sort_values("mean", ascending=False).head(5)



TÉCNICOS


,,mean
clube_id,abreviacao,
262,FLA,5.659859
284,GRE,3.877334
265,BAH,3.651795
354,CEA,3.562761
293,CAP,3.536598


In [13]:
print("CAPITÃES")
df_captains = df_fw
#df_captains = df_captains.append(df_gk)
#df_captains = df_captains.append(df_bw)
df_captains = df_captains.append(df_mf)
#df_captains = df_captains.append(df_sw)
df_captains[df_captains.home_guest == "H"].sort_values("median", ascending=False).head(5)
            

CAPITÃES


,clube_id,atleta_id,posicao_id,apelido,abreviacao,mean,median,home_guest,idx_pos,idx_round,preco_num,idx_player
8,262,87863,4,Arrascaeta,FLA,11.616667,11.60,H,2.729266,6.88689,18.05,8.553286
7,262,83257,5,Gabriel,FLA,10.793750,10.05,H,3.600144,6.88689,21.50,8.044613
168,354,96340,4,Lima,CEA,5.385714,6.60,H,2.729266,4.28355,6.70,4.980346
99,284,86757,5,Everton,GRE,7.368750,5.95,H,3.600144,4.45938,19.05,5.297941
98,284,81845,4,Alisson,GRE,5.760000,5.45,H,2.729266,4.45938,8.08,4.718003


In [14]:
df_scouts[df_scouts.atleta_id == 70360]

,rodada_id,atleta_id,clube_id,apelido,posicao_id,pontos_num,jogos_num,preco_num,abreviacao,home_guest
2124,1,70360,277,Eduardo Sasha,5,13.4,1,12.77,SAN,G
2125,2,70360,277,Eduardo Sasha,5,13.1,2,18.80,SAN,H
2126,3,70360,277,Eduardo Sasha,5,0.9,3,16.89,SAN,G
2127,4,70360,277,Eduardo Sasha,5,3.9,4,16.99,SAN,H
2128,6,70360,277,Eduardo Sasha,5,1.2,5,15.90,SAN,H
2129,7,70360,277,Eduardo Sasha,5,9.4,6,17.26,SAN,G
2130,8,70360,277,Eduardo Sasha,5,9.2,7,17.64,SAN,H
2131,9,70360,277,Eduardo Sasha,5,9.7,8,18.02,SAN,H
2132,10,70360,277,Eduardo Sasha,5,0.0,9,16.48,SAN,G
2133,11,70360,277,Eduardo Sasha,5,1.2,10,16.46,SAN,G
